<a href="https://colab.research.google.com/github/Sirisha-R-N/Transformers/blob/main/sentiment_analysis_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets accelerate peft trl

In [ ]:
!pip install evaluate

In [ ]:
!pip install -U transformers

In [ ]:
import transformers
print(transformers.__version__)


4.52.3


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
import torch
import evaluate

# Load IMDb dataset
dataset = load_dataset("imdb")
dataset = dataset.map(lambda x: {'label': int(x['label'])})  # Ensure labels are int

# Load tokenizer and tokenize
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Load model
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Configure LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "value"],  # typical for BERT
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

# Apply LoRA
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

# Evaluation metric
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return accuracy.compute(predictions=predictions, references=labels)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=1,
    load_best_model_at_end=True,
    report_to="none"
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].shuffle(seed=42).select(range(20000)),  # Subset for faster training
    eval_dataset=tokenized_datasets["test"].select(range(5000)),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train
trainer.train()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 296,450 || all params: 109,780,228 || trainable%: 0.2700


<ipython-input-3-de4e35def0b9>:63: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.285400,0.271511,0.891600
2,0.249100,0.269602,0.892000
3,0.241400,0.247202,0.903000


TrainOutput(global_step=3750, training_loss=0.2805160438537598, metrics={'train_runtime': 4796.8978, 'train_samples_per_second': 12.508, 'train_steps_per_second': 0.782, 'total_flos': 1.58413049856e+16, 'train_loss': 0.2805160438537598, 'epoch': 3.0})

In [ ]:
# Run evaluation
results = trainer.evaluate()

# Print the results
print("Evaluation results:", results)


Evaluation results: {'eval_loss': 0.24720190465450287, 'eval_accuracy': 0.903, 'eval_runtime': 153.6955, 'eval_samples_per_second': 32.532, 'eval_steps_per_second': 0.514, 'epoch': 3.0}


In [ ]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(model.device)
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=1)
    pred = torch.argmax(probs).item()
    label = "Positive" if pred == 1 else "Negative"
    return label, probs[0][pred].item()

# Example
text = "This movie was absolutely amazing!"
label, confidence = predict_sentiment(text)
print(f"Prediction: {label} (Confidence: {confidence:.2f})")

Prediction: Positive (Confidence: 0.99)


In [ ]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(model.device)
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=1)
    pred = torch.argmax(probs).item()
    label = "Positive" if pred == 1 else "Negative"
    return label, probs[0][pred].item()

# Example
text = "This movie was horrible"
label, confidence = predict_sentiment(text)
print(f"Prediction: {label} (Confidence: {confidence:.2f})")

Prediction: Negative (Confidence: 0.64)
